In [ ]:
import ipyfan
from ipyfan.utils import norm_fn, rgba2rgb
import numpy as np
import ipywidgets as widgets
import glob
from pathlib import Path
import os
import matplotlib.pyplot as plt
from ipyfan.iis_model import ritm_model as iis_model  # AI

In [ ]:
data_dir = "sentinel-2"
list_paths = sorted(glob.glob(os.path.join(data_dir, "*.png")))

In [ ]:
def load_img(path):
    return plt.imread(path)*255
    
out = ('.png', '_annotation.png')

metadata_widget = widgets.Output()

validate_button = widgets.Button(
    description='Validate',
    disabled=False)

previous_button = widgets.Button(
    description='Previous',
    disabled=False)

next_button = widgets.Button(
    description='Next',
    disabled=False)

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=len(list_paths)-1,
    description='Progress:',
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
#     style={'bar_color': 'black'},
    orientation='horizontal'
)

def validate(b):
    global w
    global curr_im
    
    in_path = Path(list_paths[progress.value])
    out_path = in_path.parent / 'annotations' / in_path.name.replace(out[0], out[1])
    out_path.parent.mkdir(exist_ok=True, parents=True)

    if np.sum(w.annI[..., :3]) > 0:  # only save if something was masked
        plt.imsave(str(out_path), w.annI)
    else:
        # Annotation is empty, delete the corresponding file if it exists
        if os.path.exists(out_path):
            os.remove(out_path)

    progress.value += 1

    curr_im = load_img(list_paths[progress.value])
    w.reset()
    w.set_image(curr_im, None)

    
def previous_image(b):
    global w
    global curr_im
    
    progress.value -= 1
    img_path = list_paths[progress.value]
    curr_im = load_img(img_path)
    
    w.reset()
    w.set_image(curr_im, None)
    
    metadata_widget.clear_output(wait=True)
    with metadata_widget:
        print(os.path.basename(img_path))
    
def next_image(b):
    global w
    global curr_im

    progress.value += 1
    img_path = list_paths[progress.value]
    curr_im = load_img(img_path)
    
    w.reset()
    w.set_image(curr_im, None)
    
    metadata_widget.clear_output(wait=True)
    with metadata_widget:
        print(os.path.basename(img_path))

        
validate_button.on_click(validate)
previous_button.on_click(previous_image)
next_button.on_click(next_image)

In [ ]:
img_path = list_paths[progress.value]
s = ipyfan.FullSegmenter(iis_model, load_img(img_path), None, 400)
w = s.w

metadata_widget.clear_output(wait=True)
with metadata_widget:
    print(os.path.basename(img_path))

widgets.VBox([s.get_widget(), 
              metadata_widget, 
              widgets.HBox([validate_button, next_button, previous_button, progress])])